<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.  
(c) Nicole Beisiegel</td>
</table>

In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

from write_hdf5 import *
from CreateSpectra import *
from bathymetry import *

# HOS: A High-Order Spectral Model for 3D Water Wave Simulations
## Creating Your Own Initial Conditions

This notebook will help you to create your own initial conditions to use with the **HOS** model. 

To get familiar with the model before that, please check out the _testsuite_ that I have created. Information on the standardized _testsuite_ can be found [here.](HOS_Visualization.ipynb)
It is also advised that, before you send a PR (pull request) @beisiegel on **GitHub** that you run the _testsuite_ to ensure that all features are still running with your changes.

This script produces initial conditions in HDF5 format for the HOS model and plots the initial sea surface elevation $\eta$, the velocity potential $\phi_{tot}$ and the bathymetry $b$ for time $t=0$.

Current options are for the ..

..**SPECTRUM**: 

- "``JONSWAP``": A Jonswap spectrum (Please specify Jonswap parameters below by editing ``JonswapParam``.)
- "``RF``": Stream function solution of Rienecker & Fenton (1981)
- "``GAUSS``": Currently under development
- "``STOKES``": Currently under development

..**BATHYMETRY**:

- "``zero``": No bathymetry
- "``constant``": Bathymetry that is constant (=base) everywhere, specify value for base in ``bathyParam``
- "``linear``": A linear sloping bathymetry with base (=base) and slope (=slope) to be specified in ``bathyParam``

In [10]:
#------------------------------------------------------------------
#-- To be altered by user

SimFolder = '/home/nicole/Development/hos/python/'
TestName  = 'test'

#-- You choose (type of spectrum and bathymetry):
spectrum = "RF"
bathymetry = "linear"

   
JonswapParam = {'g': 9.81, 'gamma': 3.3, 'alpha_p': 0.081, 'dir_spr': 40.*np.pi/180., 'kp': 0.033, \
                'lmbda': 2*np.pi/0.033,'omega_p': np.sqrt(9.81*0.033), 'epsilon': 0.12, 'theta_p': 0.0,\
                'h': 1000., 'sigma': 0.06/0.033, 'hs': (0.24/0.033)}

GaussParam = {}
StokesParam ={}

#-- Set your parameters here
parameters = JonswapParam  
#parameters = GaussParam
#parameters = StokesParam

#-- Set bathymetry config here
bathyParam = {'base': .20, 'slope': 0.01 }
    
#-- Labels for the simulation run
saveflg  = 1
runsubid = 1

#-- Discretization: Number of cells (fourier modes) in x and y direction
alpha = 7
Nx    = 2**alpha  
Ny    = Nx

#-- Length of the domain
Lx  = 64*parameters['lmbda']  
Ly  = Lx                

#-- Time parameters
Tp     = 1  
T      = 150 * Tp   
dtsave = 1.0

#-- Filtering Parameter
Tramp  = 200

#------------------------------------------------------------------
#-- Only internal subroutines follow

#-- Wavenumber domain
kx = 2*np.pi/Lx*np.arange(-Nx/2,Nx/2)
ky = 2*np.pi/Ly*np.arange(-Ny/2,Ny/2)

#-- The options for the different spectra
switch = {'JONSWAP': create_jonswap,
          'RF': create_rieneckerfenton,
          'Gauss': create_gauss,
          'Stokes': create_stokes}

if spectrum in switch:
     etar, phir = switch[spectrum](kx,ky,Param = parameters)
else:
     pass

bathy = create_bathymetry(kx,ky, Type = bathymetry, Param = bathyParam)

#-- Plot initial condition
plt.figure(1)
plt.suptitle("Initial Condition TYPE: "+str(spectrum),fontsize=18)

plt.subplot(221)
plt.pcolormesh(kx,ky,etar, cmap=plt.get_cmap("Blues"))
plt.colorbar()
plt.title("Initial Sea Surface Elevation $\eta$")
plt.xlabel("kx")
plt.xlim(np.amin(kx), np.amax(kx))
plt.ylabel("ky")
plt.ylim(np.amin(ky), np.amax(ky))

plt.subplot(222)
plt.pcolormesh(kx,ky,bathy, cmap=plt.get_cmap("Greys"))
plt.colorbar()    
plt.title("Bathymetry $b$")
plt.xlabel("kx")
plt.xlim(np.amin(kx), np.amax(kx))
plt.ylabel("ky")
plt.ylim(np.amin(ky), np.amax(ky))

plt.subplot(223)
plt.pcolormesh(kx,ky,phir, cmap=plt.get_cmap("Greens"))
plt.colorbar()
plt.title("Initial Velocity Potential $\phi_{tot}$")
plt.xlabel("kx")
plt.xlim(np.amin(kx), np.amax(kx))
plt.ylabel("ky")
plt.ylim(np.amin(ky), np.amax(ky))

plt.tight_layout()
plt.show()

#-- Write information
write_hdf5(SimFolder, TestName, Nx, Ny, Lx, Ly, parameters['g'], parameters['h'], T, dtsave, saveflg, runsubid, \
           Tramp, etar, phir, bathy) 

Success!
Output files written.
